# Login

Refresh Token (09/03/2025): 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3NDE1MTM2MDYsImV4cCI6MTc0Mjc3NjI0NywibmJmIjoxNzQxNTEzNjA2LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQUFBQm56V09HUnFmYVpJdUU1RS1LLXRRN1JMaUdtRzJxdjhCWm83NUROWVBkamVCWGtsNDNELXg0U0o4VkpKR3U3YnpqYl9IRGJHc3lVVWdza1pDZHhSa2NIMXFYTTh6cXdsRUZCVEZRSm83SnR3SEt6QkE9IiwiZGlzcGxheV9uYW1lIjoiU1VQUklZTyBCSEFUVEFDSEFSWUEiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJhMmYzYzZmYjRkMzY3MmZhN2M0MTI5YmFhZmI5NDE3YjI0YjU5NTkxOTU3ZmYyYmU4NjQ2MTM1OCIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWVMzNTg3NSIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.FOiTCPcLGpCkximPLw3e3f0yfsxXKCtQviqMUVlDFXw'


For LTP initiate TradeManage -> will initiate websocket -> tm.fetchltp()

In [ ]:
import login
print(login.get_authorization_code())

In [ ]:
import pandas as pd

config_df = pd.read_csv("cred.csv")
config = config_df.iloc[0].to_dict()
refresh_token = config["refresh_token"]

login.get_access_token(refresh_token,login_type="refresh_token")

In [ ]:
#Define Trade Symbol and QTY

symbol="MCX:CRUDEOILM25MARFUT"
qty=1
mode="Auto"      #Use Manual for Debugging
MaxLoss=-500

# Fetch Historical data(also SuperTrend and other parameters calculations) and Start Websocket

In [ ]:
#Continuously updating historical data df_5m and df_30m for calculations

import history
import schedule
import threading
import time

# === Step 1: Fetch Historical Data Before Market Starts ===
df_60m = history.fetch_historical_data(symbol, "60")

print("Initial data loaded:")
print("df_60m:", df_30m.shape)

def update_df_60m():
    global df_30m
    print("Updating df_60m at", time.strftime("%H:%M:%S"))
    df_60m = history.fetch_historical_data(symbol, "60")
    print("df_30m updated:", df_30m.shape)


schedule.every(30).minutes.do(update_df_60m)

def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(300)

# === Step 4: Start the Scheduler in a Background Thread ===
scheduler_thread = threading.Thread(target=run_scheduler, daemon=True)
scheduler_thread.start()

print("Scheduler for updating df_60m started.")


# Logic implementation and Trading

In [ ]:
import threading
import time
from TradeLogic2 import signal   #Importing trading algorithm

decision = None 

def signal_loop():
    global decision
    while True:
        # decision = signal(df_5m, df_30m)
        decision=signal(df_60m)
        time.sleep(300)
# Run the signal loop in a background thread
signal_thread = threading.Thread(target=signal_loop, daemon=True)
signal_thread.start()

#Continuos signal generation

In [ ]:
from TradeAction import TradeManager  

tm = TradeManager(symbol,qty,mode,MaxLoss)  

def TradeEngine(MaxLoss=-500, decision=None, mode="Auto"): 
    
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    try:
        current_mtm = tm.mtm()
        if current_mtm is None:
            print("Error in MTM calculation")
            return "MTM calculation error"

        if current_mtm > MaxLoss:
            print('Trading started')

            if decision == 1:
                if mode=="Manual":
                    print(f"[{current_time}] 📢 Take a LONG position")
                    return "Long signal generated"
                elif mode=="Auto":
                    result = tm.longentry(tm.position, tm.SL_order, tm.spl_last_pos, MaxLoss)
                    if result == "Daily Stop Loss reached":
                        return result
                        
            elif decision == -1:
                if mode=="Manual":
                    print(f"[{current_time}] 📢 Take a SHORT position")
                    return "Short signal generated"
                elif mode=="Auto":
                    result = tm.shortentry(tm.position, tm.SL_order, tm.spl_last_pos, MaxLoss)
                    if result == "Daily Stop Loss reached":
                        return result
            else:
                print("No clear trading signal")
                return "No clear trading signal"

        else:
            print("MaxLoss Breached")
            return "MaxLoss Breached"

    except Exception as e:
        print(f"Error in TradeEngine: {e}")
        # Emergency exit for all open positions
        tm.emergency_exit()


# Press "I" twice to stop trading

In [ ]:
# Start Trading 
from datetime import datetime  

while True:
    try:
        result=TradeEngine(MaxLoss,decision,mode)
        print(f"Trade decision: {result}")
        if result=="MTM calculation error" or result=="Daily Stop Loss reached" or result=="MaxLoss Breached":
            print("Trade engine stopping due to....",result)
            tm.emergency_exit()
            break
        time.sleep(5)
        
    except KeyboardInterrupt:
        print("🚨 Trading session manually stopped.")
        break  

    except Exception as e:
        print(f"❌ Error in trading loop: {e}")
        break 


In [ ]:
# import csv
# import time
# from datetime import datetime  

# # Define CSV file path
# csv_file = "trade_results.csv"

# # Ensure the CSV has headers if it does not exist
# with open(csv_file, mode='a', newline='') as file:
#     writer = csv.writer(file)
#     if file.tell() == 0:  # If file is empty, write headers
#         writer.writerow(["Timestamp", "Trade Decision"])

# while True:
#     try:
#         result = TradeEngine(MaxLoss, decision, mode)
#         print(f"Trade decision: {result}")

#         # Append result to CSV
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), result])

#         # Stop if certain conditions are met
#         if result in ["MTM calculation error", "Daily Stop Loss reached", "MaxLoss Breached"]:
#             print("Trade engine stopping due to....", result)
#             tm.emergency_exit()
#             break

#         time.sleep(5)

#     except KeyboardInterrupt:
#         print("🚨 Trading session manually stopped.")
#         break  

#     except Exception as e:
#         print(f"❌ Error in trading loop: {e}")
#         break  


In [ ]:
import sys
print(sys.executable)

In [ ]:
from graphviz import Digraph

# Create a directed graph
dot = Digraph(format='png')

# Nodes for data source and preprocessing steps
dot.node('A', 'data_source - 11 col')
dot.node('B', 'DateTime')
dot.node('C', 'Numeric')
dot.node('D', 'ModeCatImputer-StringCast-DateTimeTransformer')
dot.node('E', 'MeanImputer')
dot.node('F', 'StandardScalerWrapper')
dot.node('G', 'RandomForestClassifier')

# Connect nodes to show the flow
dot.edge('A', 'B', label='1 col')
dot.edge('A', 'C', label='10 col')
dot.edge('B', 'D')
dot.edge('C', 'E')
dot.edge('D', 'F', label='10 col')
dot.edge('E', 'F', label='1 col')
dot.edge('F', 'G')

# Save and render the graph
dot.render('flowchart')
